In [ ]:
import socket
import time
import threading
import dash
from dash import html, dcc, Input, Output, State, callback_context
import plotly.graph_objects as go
from collections import deque
import dash_daq as daq

monitor_thread_running = False
client_socket = None
message_str_list = []
response_str_list = []
response_str_list_2 = []
response_str_list_3 = []
response_str_list_4 = []
response_str_list_5 = []
response_str_list_6 = []
response_str_list_7 = []
response_str_list_8 = []

response_dict = {}
X = deque(maxlen=20)
X.append(1)
Y = deque(maxlen=20)
Y.append(1)
Y_2 = deque(maxlen=20)
Y_2.append(1)
Y_3 = deque(maxlen=20)
Y_3.append(1)
Y_4 = deque(maxlen=20)
Y_4.append(1)
Y_5 = deque(maxlen=20)
Y_5.append(1)

# Plotly grafiği oluştur
fig = go.Figure(go.Indicator(
    mode="gauge+number",
    value=450,
    title={'text': "Speed"},
    delta={'reference': 300, 'increasing': {'color': "blue"}},
    gauge={
        'bar': {'color': "blue"},
        'axis': {'range': [None, 500]},
        'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 490}
    }
))
fig.update_layout(
    plot_bgcolor='white',  # Arka plan rengi
    margin=dict(l=5, r=5, t=5, b=0),
    paper_bgcolor='#DDF3F6',
    width=400,  # Genişlik 800 piksel
    height=280  # Yükseklik 600 piksel  # Kağıt rengi (grafik dışındaki alan)
)

# Uygulama başlat
app = dash.Dash(__name__)

# Uygulama düzeni
app.layout = html.Div(
    children=[
        html.Div(
            id="baslik",
            children=[html.H1("STM32 CONTROL PANEL", style={'color': '#00008B', 'text-align': 'center'})],
            style={'width': '1400px', 'height': '60px', 'background-color': '#DDF3F6', 'margin-bottom': '10px'}),
        html.Div(
            children=[
                html.Div(id='sol_div',
                         children=[
                             html.Div([
                                 html.Label("IP ADRESİ",
                                            style={'color': "#fec036", "margin-left": "20px", "margin-bottom": "10px"}),
                                 html.Label("PORT",
                                            style={'color': "#fec036", "margin-left": "110px", "margin-right": "30px",
                                                   "margin-bottom": "15px"}),
                                 dcc.Input(id='ip-input', type='text', value='',
                                           style={'color': "#fec036", 'height': '25px', 'width': '150px',
                                                  "margin-left": "5px"}),

                                 dcc.Input(id='port-input', type='number', value='',
                                           style={'color': "#fec036", 'height': '25px', 'width': '157px',
                                                  "margin-left": "5px",'margin-top':'10px'})
                             ], style={'background-clor':'#CCE5FF', 'width': '400px', "height": "60px",
                                       'display': 'inline-block', "margin-top": "20px"}),
                             html.Div([
                                  html.Button('Connect', id='connect-button', n_clicks=0, style={"color": "#fec036",'height': '30px', 'width': '250px', "margin-left": "5px", "margin-top": "20px"}),
                                  html.Button('Disconnect', id='disconnect-button', n_clicks=0, style={"color": "#fec036",'height': '30px', 'width': '250px', "margin-left": "5px", "margin-top": "20px"})     
                             ], style={'background-color': '#DDF3F6', 'width': '400px', "height": "35px",
                                       'display': 'inline-block'}),
                             html.Div(id='output-state'),
                             html.Div([
                                 dcc.Graph(id='gauge-graph', figure=fig),
                                 dcc.Interval(
                                     id='interval-component_a2',
                                     interval=1000,  # Her saniyede bir güncelle
                                     n_intervals=0
                                 )
                             ], style={'background-color': '#DDF3F6', 'width': '405px', "height": "150px",
                                       'display': 'inline-block', "margin-top": "150px"}),
                             html.Div([
                                 daq.Thermometer(
                                     id='thermometer',
                                     min=0,
                                     max=100,
                                     value=50,
                                     showCurrentValue=True,
                                     units="C"
                                 ),
                                 dcc.Interval(
                                     id='interval-component_a1',
                                     interval=2000,  # Her iki saniyede bir güncelle
                                     n_intervals=0
                                 )
                             ], style={'background-color': '#DDF3F6', 'width': '400px', "height": "100px",
                                       'display': 'inline-block', "margin-top": "300px"}),

                             html.Div([
                                 daq.Gauge(
                                     id="gauge",
                                     color={"gradient": True,
                                            "ranges": {"green": [0, 6], "yellow": [6, 8], "red": [8, 10]}},
                                     value=2,
                                     label='Default',
                                     max=10,
                                     min=0, ),
                                dcc.Interval(
                                    id='interval-component_6',
                                    interval=2*1000,  # milliseconds
                                    n_intervals=0)

                             ], style={'background-color': '#DDF3F6', 'width': '400px', "height": "100px",
                                       'display': 'inline-block', "margin-top": "220px"}),

                         ],
                         style={'width': '450px', 'height': '1488px', 'background-color': '#DDF3F6', 'float': 'left',
                                'margin-left': '40px'}),
                html.Div(id='sag_div',
                         children=[
                             html.Div([
                                 dcc.Tabs(id="tabs-example-graph", value='', children=[
                                     dcc.Tab(label='VOLTAGE', value='', children=[dcc.Graph(id='graph-vol'),
                                                                                  dcc.Interval(
                                                                                      id='interval-component',
                                                                                      interval=1 * 1000,
                                                                                      # Her saniyede bir güncelle
                                                                                      n_intervals=0)
                                                                                  ]),
                                     dcc.Tab(label='CURRENT', value='', children=[dcc.Graph(id='graph-cur'),
                                                                                  dcc.Interval(
                                                                                      id='interval-component_2',
                                                                                      interval=1 * 1000,
                                                                                      # Her saniyede bir güncelle
                                                                                      n_intervals=0)
                                                                                  ]),
                                     dcc.Tab(label='SPEED', value='', children=[dcc.Graph(id='graph-speed'),
                                                                                  dcc.Interval(
                                                                                      id='interval-component_3',
                                                                                      interval=1 * 1000,
                                                                                      # Her saniyede bir güncelle
                                                                                      n_intervals=0)]),
                                     dcc.Tab(label='THERMO', value='', children=[dcc.Graph(id='graph-thermo'),
                                                                                  dcc.Interval(
                                                                                      id='interval-component_4',
                                                                                      interval=1 * 1000,
                                                                                      # Her saniyede bir güncelle
                                                                                      n_intervals=0)]),
                                     dcc.Tab(label='TORQUE', value='',children=[dcc.Graph(id='graph-torque'),
                                                                                  dcc.Interval(
                                                                                      id='interval-component_5',
                                                                                      interval=1 * 1000,
                                                                                      # Her saniyede bir güncelle
                                                                                      n_intervals=0)])

                                 ]),
                                 html.Div(id='tabs-content-example-graph')
                             ],
                                 style={'width': '750px', 'height': '1000', 'background-color': '#DDF3F6', 'float': 'right',
                                        'margin-left': '100px', 'margin-right': '40px'}),
                             html.Div([
                                 html.Pre(id='monitor-read-display', style={"white-space": "pre-wrap", "word-wrap": "break-word", "overflow-y": "scroll", "overflow-x": "auto"})
                             ],
                                 style={'width': '750px', 'height': '1000px', 'background-color': '#DDF3F6',
                                        'float': 'left',
                                        'margin-left': '197px', 'overflow-y': 'auto'}),

                         ])

            ],
            style={'display': 'flex', 'flex-direction': 'row','background-color': '#6AC9EC'}
        )

    ],
    style={'display': 'flex', 'flex-direction': 'column','height': '1100','width':'1000','background-color': '#6AC9EC'}
)


def send_read_message(client_socket, addr):
    message = "READ*{}".format(addr)
    client_socket.sendall(message.encode())
    print(f"Sending read message: {message}")
    response = client_socket.recv(1024).decode()
    return response


def read_data(addr):
    response = send_read_message(client_socket, addr)
    if response:
        if addr == 23:
            response_str_list.append(int(response))
        elif addr == 24:
            response_str_list_2.append(int(response))
        elif addr == 25:
            response_str_list_3.append(int(response))
        elif addr == 26:
            response_str_list_4.append(int(response))
        elif addr == 27:
            response_str_list_5.append(int(response))
        elif addr == 28:
            response_str_list_6.append(int(response))
        elif addr == 29:
            response_str_list_7.append(int(response))
        elif addr == 30:
            response_str_list_8.append(int(response))


def main(server_ip, server_port):
    global client_socket
    try:
        # Client socket oluştur
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
            client_socket.connect((server_ip, server_port))
            print("Connected to the server.")

            while True:
                read_data(23)
                print(response_str_list)
                read_data(24)
                print(response_str_list_2)
                read_data(25)
                print(response_str_list_3)
                read_data(26)
                print(response_str_list_4)
                read_data(27)
                print(response_str_list_5)
                read_data(28)
                print(response_str_list_6)
                read_data(29)
                print(response_str_list_7)
                read_data(30)
                print(response_str_list_8)
                time.sleep(1)  # Her b

    except Exception as e:
        print("Error:", e)


@app.callback(
    [Output('output-state', 'children')],
    [Input('connect-button', 'n_clicks'),
     Input('disconnect-button', 'n_clicks')],
    [State('ip-input', 'value'),
     State('port-input', 'value')]
)
def send_command_to_server(connect_clicks, disconnect_clicks, ip_value, port_value):
    ctx = callback_context
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    style = {'color': '#fec036'}

    if button_id == 'connect-button' and connect_clicks is not None and connect_clicks > 0:
        if ip_value and port_value:
            port_value_sck = int(port_value)
            threading.Thread(target=main, args=(ip_value, port_value_sck)).start()
            return ("Connection established.")
        else:
            return ("Please enter IP address and port number.")
    elif button_id == 'disconnect-button' and disconnect_clicks is not None and disconnect_clicks > 0:
        client_socket.close()
        return ("Connection terminated.",)
    return (None,)



# monitor-read-display güncellemesi için callback fonksiyonu
@app.callback(
    Output('monitor-read-display', 'children'),
    [Input('interval-component', 'n_intervals')]
)
def update_monitor_read_display(n):
    combined_text = ""
    for addr, response_list in zip(range(23, 31), [response_str_list, response_str_list_2, response_str_list_3, response_str_list_4, response_str_list_5, response_str_list_6, response_str_list_7, response_str_list_8]):
        if response_list:
            combined_text += f"Address {addr}:\n"
            combined_text += "\n".join([f"Received message: {response}" for response in response_list])
            combined_text += "\n"
    return combined_text if combined_text else "No data"


@app.callback(
    Output('graph-vol', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def update_indicator_graph(n):
    global X
    global Y
    if response_str_list:  # Eğer response_str_list boş değilse devam et
        X.append(X[-1] + 1)
        Y.append(response_str_list[-1])
        trace1 = go.Scatter(
            x=list(X),
            y=list(Y),
            name='temperature_value',
            mode='lines+markers'
        )
        layout = go.Layout(xaxis=dict(range=[min(X), max(X)]),
                           yaxis=dict(range=[0, 60]))
        return {'data': [trace1], 'layout': layout}
    else:  # response_str_list boşsa grafik güncellemesini engelle
        raise dash.exceptions.PreventUpdate



@app.callback(
    Output('graph-cur', 'figure'),
    [Input('interval-component_2', 'n_intervals')]
)
def update_indicator_graph_2(n):
    global X
    global Y_2
    if response_str_list_2:  # Eğer response_str_list boş değilse devam et
        X.append(X[-1] + 1)
        Y_2.append(response_str_list_2[-1])
        trace2 = go.Scatter(
            x=list(X),
            y=list(Y_2),
            name='temperature_value',
            mode='lines+markers'
        )
        layout = go.Layout(xaxis=dict(range=[min(X), max(X)]),
                           yaxis=dict(range=[0, 60]))
        return {'data': [trace2], 'layout': layout}
    else:  # response_str_list boşsa grafik güncellemesini engelle
        raise dash.exceptions.PreventUpdate




@app.callback(
    Output('graph-speed', 'figure'),
    [Input('interval-component_3', 'n_intervals')]
)
def update_indicator_graph_3(n):
    global X
    global Y_3
    if response_str_list_3:  # Eğer response_str_list boş değilse devam et
        X.append(X[-1] + 1)
        Y_3.append(response_str_list_3[-1])
        trace3 = go.Scatter(
            x=list(X),
            y=list(Y_3),
            name='temperature_value',
            mode='lines+markers'
        )
        layout = go.Layout(xaxis=dict(range=[min(X), max(X)]),
                           yaxis=dict(range=[0, 60]))
        return {'data': [trace3], 'layout': layout}
    else:  # response_str_list boşsa grafik güncellemesini engelle
        raise dash.exceptions.PreventUpdate
@app.callback(
    Output('graph-thermo', 'figure'),
    [Input('interval-component_4', 'n_intervals')]
)
def update_indicator_graph_4(n):
    global X
    global Y_4
    if response_str_list_4:  # Eğer response_str_list boş değilse devam et
        X.append(X[-1] + 1)
        Y_4.append(response_str_list_4[-1])
        trace4 = go.Scatter(
            x=list(X),
            y=list(Y_4),
            name='temperature_value',
            mode='lines+markers'
        )
        layout = go.Layout(xaxis=dict(range=[min(X), max(X)]),
                           yaxis=dict(range=[0, 60]))
        return {'data': [trace4], 'layout': layout}
    else:  # response_str_list boşsa grafik güncellemesini engelle
        raise dash.exceptions.PreventUpdate

@app.callback(
    Output('graph-torque', 'figure'),
    [Input('interval-component_5', 'n_intervals')]
)
def update_indicator_graph_5(n):
    global X
    global Y_5
    if response_str_list_5:  # Eğer response_str_list boş değilse devam et
        X.append(X[-1] + 1)
        Y_5.append(response_str_list_5[-1])
        trace5 = go.Scatter(
            x=list(X),
            y=list(Y_5),
            name='temperature_value',
            mode='lines+markers'
        )
        layout = go.Layout(xaxis=dict(range=[min(X), max(X)]),
                           yaxis=dict(range=[0, 60]))
        return {'data': [trace5], 'layout': layout}
    else:  # response_str_list boşsa grafik güncellemesini engelle
        raise dash.exceptions.PreventUpdate

@app.callback(
    Output('gauge', 'value'),
    [Input('interval-component_6', 'n_intervals')]
)
def update_gauge(n):
    # Güncellenmiş bir value almak için listenin son elemanını kullan
    if response_str_list_8:
        return response_str_list_8[-1]
    else:
        return 0  # Liste boşsa varsayılan değeri döndür



@app.callback(
    Output('thermometer', 'value'),
    [Input('interval-component_a1', 'n_intervals')]
)
def update_thermometer(n):
    # Güncellenmiş bir value almak için listenin son elemanını kullan
    if response_str_list_7:
        return response_str_list_7[-1]
    else:
        return 100  # Liste boşsa varsayılan değeri döndü


@app.callback(
    Output('gauge-graph', 'figure'),
    [Input('interval-component_a2', 'n_intervals')]
)
def update_graph(n):
    # Eğer response_str_list_6 listesi boşsa veya değer içermiyorsa işlem yapılmayacak
    if response_str_list_6:
        # response_str_list_6 listesinin son değerini al
        new_value = response_str_list_6[-1]

        # Grafik figürünü güncelle
        fig.update_traces(value=new_value)

    return fig
if __name__ == "__main__":
    app.run_server(port=8214)


Dash is running on http://127.0.0.1:8214/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8214/ (Press CTRL+C to quit)
127.0.0.1 - - [16/May/2024 22:44:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:02] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:02] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:02] "GET /_favicon.ico?v=2.4.0 HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:02] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:02] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:02] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [16/Ma

Connected to the server.
Sending read message: READ*23
[10]
Sending read message: READ*24
[20]
Sending read message: READ*25
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint]

127.0.0.1 - - [16/May/2024 22:44:07] "POST /_dash-update-component HTTP/1.1" 500 -


[40]
Sending read message: READ*27
[60]
Sending read message: READ*28
[70]
Sending read message: READ*29
[80]
Sending read message: READ*30
[5]


...

Sending read message: READ*23
[10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31]
Sending read message: READ*24
[20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35]
Sending read message: READ*25
[62, 43, 30, 62, 43, 30, 62, 43, 30, 62, 43, 30, 62, 43, 30, 62, 43, 30, 62, 43, 30, 62, 43, 30, 62, 43, 30, 62, 43, 30, 62, 43, 30, 62, 43, 30, 62, 43]
Sending read message: READ*26
[40, 19, 35, 40, 19, 35, 40, 19, 35, 40, 19, 35, 40, 19, 35, 40, 19, 35, 40, 19, 35, 40, 19, 35, 40, 19, 35, 40, 19, 35, 40, 19, 35, 40, 19, 35, 40, 19]
Sending read message: READ*27
[60, 85, 50, 60, 85, 50, 60, 85, 50, 60, 85, 50, 60, 85, 50, 60, 85, 50, 60, 85, 50, 60, 85, 50, 60, 85, 50, 60, 85, 50, 60, 85, 50, 60, 85, 50, 60, 85]
Sending read message: READ*28
[70, 42, 55, 70, 42, 55, 70, 42, 55, 70, 42, 55, 70, 42

127.0.0.1 - - [16/May/2024 22:44:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:48] "POST /_dash-update-component HTTP/1.1" 200 -


Sending read message: READ*23
[10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45, 10, 31, 45]
Sending read message: READ*24
[20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 42, 20, 35, 35]
Sending read message: READ*25
Error: [WinError 10054] Varolan bir bağlantı uzaktaki bir ana bilgisayar tarafından zorla kapatıldı


127.0.0.1 - - [16/May/2024 22:44:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:44:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [16/May/2024 22:45:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:45:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:45:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:45:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:45:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:45:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:45:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:45:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:45:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:45:35] "POST /_dash-update-component HTTP/1.1" 200 -
